In [3]:
import sys

# parse_s4ext
extensions_index_script_dir = '/home/jcfr/Projects/SlicerExtensionsIndex/scripts/'
if extensions_index_script_dir not in sys.path:
    sys.path.insert(0, extensions_index_script_dir)

from check_description_files import parse_s4ext

# ExtensionProject
slicer_scripts_dir = "/tmp/SlicerWizard/Utilities/Scripts/"
if slicer_scripts_dir not in sys.path:
    sys.path.insert(0, slicer_scripts_dir)

from SlicerWizard.ExtensionProject import ExtensionProject

In [4]:
import errno
import os
    
# Copied from https://github.com/scikit-build/scikit-build/blob/master/skbuild/utils/__init__.py
def mkdir_p(path):
    """Ensure directory ``path`` exists. If needed, parent directories
    are created.
    Adapted from http://stackoverflow.com/a/600612/1539918
    """
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:  # pragma: no cover
            raise

In [5]:
import urllib.request

from socket import timeout as SocketTimeout

def check_url(url, timeout=1):
    try:
        return urllib.request.urlopen(url, timeout=timeout).getcode(), None
    except urllib.request.HTTPError as exc:
        return exc.code, str(exc)
    except (TimeoutError, urllib.request.URLError, SocketTimeout) as exc:
        return -1, str(exc)

In [25]:
def download_cmakelists(scmurl, dest_filepath):
    scmurl = scmurl.replace("https://github.com/", "").replace("git://github.com/", "").replace(".git", "")
    user, repo = scmurl.split("/")
    cmakelists_url = "https://raw.githubusercontent.com/{user}/{repo}/{revision}/CMakeLists.txt".format(
        user=user,
        repo=repo,
        revision=metadata['scmrevision']
    )
    if not os.path.exists(dest_filepath):
        try:
            with urllib.request.urlopen(cmakelists_url) as response, open(dest_filepath, 'wb') as out_file:
                shutil.copyfileobj(response, out_file)
        except urllib.request.HTTPError as exc:
            return [(None, "%d, %s" % (exc.code, cmakelists_url),)]
    
    return []

In [66]:
import urllib.parse

def check_cmakelists(filepath):
    cmakelists_dir = os.path.dirname(filepath)
    cmakelists_filename = os.path.basename(filepath)

    project = ExtensionProject(cmakelists_dir, filename=cmakelists_filename)
    
    urls = []
    errors = []

    def _collect_urls(attribute, required=False, split=False):
        try:
            cmake_variable_name = "EXTENSION_" + attribute.upper()
            url = project.getValue(cmake_variable_name, substitute=True).strip()
        except KeyError as exc:
            if not required:
                return
            message = str(exc)
            errors.append((attribute, message,))
            return

        if not url:
            if required:
                message = "script sets %r to an empty value" % cmake_variable_name,
                errors.append((attribute, message))
            return

        try:
            result = urllib.parse.urlparse(url)
            if not all([result.scheme, result.netloc, result.path]):
                raise ValueError("failed to parse either scheme, netloc or path")
        except ValueError:
            message = "script sets %r to an invalid url [%s]" % (cmake_variable_name, url),
            errors.append((attribute, message))
            return

        if split:
            for item in url.split(" "):
                urls.append((attribute, url,))
        else:
            urls.append((attribute, url,))
    
    _collect_urls("homepage", required=True)
    _collect_urls("iconurl", required=True)
    _collect_urls("screenshoturls", split=True)

    for attribute, url in urls:
        code, error = check_url(url)
        if code != 200:
            errors.append((attribute, "%d, %s, %s" % (code, url, error),))
    
    return errors

In [67]:
import glob
import shutil

import urllib.request

s4ext_dir = "/home/jcfr/Projects/SlicerExtensionsIndex"

s4ext_filepaths = glob.glob(os.path.join(s4ext_dir, "*.s4ext"))

for index, filepath in enumerate(s4ext_filepaths):
    extension_name = os.path.splitext(os.path.basename(filepath))[0]
    
    msg = "[%d/%d] Checking %s ... " % (index + 1, len(s4ext_filepaths), extension_name)
    print(msg.ljust(50), end='')
    
    metadata = parse_s4ext(filepath)
    scmurl = metadata['scmurl']
    
    if 'github' not in scmurl:
        print("[skipping, not a GitHub repository]")
        print("")
        continue
    
    errors = []
    
    # Directory to store extension CMakeLists.txt files
    dest_dir = "Extensions-CMakeLists"
    mkdir_p(dest_dir)
    
    cmakelists_filepath = os.path.join(dest_dir, '%s-CMakeLists.txt' % extension_name)

    # Download CMakeLists.txt
    errors.extend(download_cmakelists(scmurl, cmakelists_filepath))

    # Check CMakeLists.txt
    if os.path.exists(cmakelists_filepath):
       errors.extend(check_cmakelists(cmakelists_filepath))

    if errors:
        print("[errors]")
        for context, error in errors:
            if context is None:
                print("  %s" % error)
            else:
                print("  %s, %s" % (context, error))
        print("")
        continue
    else:
        print("[ok]")

    print("")
    

[1/182] Checking DiceComputation ...              [ok]

[2/182] Checking QuantitativeReporting ...        [errors]
  homepage, 403, https://qiicr.gitbooks.io/quantitativereporting-guide, HTTP Error 403: Forbidden
  iconurl, 404, https://www.slicer.org/w/images/3/30/QuantitativeReportingLogo.png, HTTP Error 404: Not Found
  screenshoturls, 404, https://www.slicer.org/w/images/f/fe/QuantitativeReporting-screenshot.jpg, HTTP Error 404: Not Found

[3/182] Checking CarreraSlice ...                 [errors]
  homepage, "script does not set 'EXTENSION_HOMEPAGE'"
  iconurl, "script does not set 'EXTENSION_ICONURL'"

[4/182] Checking MFSDA ...                        [errors]
  screenshoturls, 400, https://raw.githubusercontent.com/DCBIA-OrthoLab/MFSDA_Python/master/MFSDA/Resources/Icons/RunMFSDA.png https://raw.githubusercontent.com/DCBIA-OrthoLab/MFSDA_Python/master/MFSDA/Resources/Icons/SelectVariables.png, HTTP Error 400: Bad Request
  screenshoturls, 400, https://raw.githubusercontent.com/D

[27/182] Checking SequenceRegistration ...        [errors]
  screenshoturls, 400, https://raw.githubusercontent.com/moselhy/SlicerSequenceRegistration/master/screenshot01.png https://raw.githubusercontent.com/moselhy/SlicerSequenceRegistration/master/screenshot02.png, HTTP Error 400: Bad Request
  screenshoturls, 400, https://raw.githubusercontent.com/moselhy/SlicerSequenceRegistration/master/screenshot01.png https://raw.githubusercontent.com/moselhy/SlicerSequenceRegistration/master/screenshot02.png, HTTP Error 400: Bad Request

[28/182] Checking SlicerRadiomics ...             [errors]
  iconurl, 404, http://www.slicer.org/w/images/6/6b/RadiomicsExtension.png, HTTP Error 404: Not Found
  screenshoturls, 404, http://www.slicer.org/w/images/3/3b/SlicerRadiomics-lung.png, HTTP Error 404: Not Found

[29/182] Checking OsteotomyPlanner ...            [errors]
  iconurl, 404, https://raw.githubusercontent.com/KitwareMedical/OsteotomyPlanner/master/Planner/Resources/Icons/Planner.png, HTTP E

[54/182] Checking GyroGuide ...                   [ok]

[55/182] Checking PETTumorSegmentation ...        [ok]

[56/182] Checking IASEM ...                       [ok]

[57/182] Checking Auto3dgm ...                    [errors]
  homepage, ("script sets 'EXTENSION_HOMEPAGE' to an invalid url [https://toothandclaw.github.io]",)

[58/182] Checking ScatteredTransform ...          [ok]

[59/182] Checking MONAILabel ...                  [errors]
  homepage, "script does not set 'EXTENSION_HOMEPAGE'"
  iconurl, "script does not set 'EXTENSION_ICONURL'"

[60/182] Checking SlicerWMA ...                   [errors]
  screenshoturls, 404, http://www.example.com/Slicer/Extensions/SlicerWMA/Screenshots/1.png, HTTP Error 404: Not Found

[61/182] Checking SliceTracker ...                [errors]
  iconurl, 404, https://www.slicer.org/w/images/b/b1/SliceTracker_Logo_1.1_128x128.png, HTTP Error 404: Not Found

[62/182] Checking SNRMeasurement ...              [errors]
  homepage, 404, http://www.slicer.

[83/182] Checking SegmentationAidedRegistration ... [ok]

[84/182] Checking ChangeTracker ...               [ok]

[85/182] Checking BoneReconstructionPlanner ...   [errors]
  screenshoturls, 400, https://raw.githubusercontent.com/SlicerIGT/SlicerBoneReconstructionPlanner/main/BoneReconstructionPlanner/Resources/Screenshots/screenshotPlanning.png https://raw.githubusercontent.com/SlicerIGT/SlicerBoneReconstructionPlanner/main/BoneReconstructionPlanner/Resources/Screenshots/screenshotPatientSpecificSurgicalGuides.png, HTTP Error 400: Bad Request
  screenshoturls, 400, https://raw.githubusercontent.com/SlicerIGT/SlicerBoneReconstructionPlanner/main/BoneReconstructionPlanner/Resources/Screenshots/screenshotPlanning.png https://raw.githubusercontent.com/SlicerIGT/SlicerBoneReconstructionPlanner/main/BoneReconstructionPlanner/Resources/Screenshots/screenshotPatientSpecificSurgicalGuides.png, HTTP Error 400: Bad Request

[86/182] Checking ZFrameRegistration ...          [errors]
  screenshotu

[107/182] Checking GelDosimetryAnalysis ...       [errors]
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/80/20150605_Gamma_98percent.png https://www.slicer.org/slicerWiki/images/9/96/GelDosimetry_Step3a_OpticalCtMeasuredDose.png http://www.slicer.org/slicerWiki/images/6/68/GelDosimetrySlicelet_0.1_CalibrationCurvesAligned.png http://www.slicer.org/slicerWiki/images/5/51/GelDosimetrySlicelet_0.1_OdVsDoseCurve.png, HTTP Error 404: Not Found
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/80/20150605_Gamma_98percent.png https://www.slicer.org/slicerWiki/images/9/96/GelDosimetry_Step3a_OpticalCtMeasuredDose.png http://www.slicer.org/slicerWiki/images/6/68/GelDosimetrySlicelet_0.1_CalibrationCurvesAligned.png http://www.slicer.org/slicerWiki/images/5/51/GelDosimetrySlicelet_0.1_OdVsDoseCurve.png, HTTP Error 404: Not Found
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/80/20150605_Gamma_98percent.png https://www.slicer.org/slicerWiki/i

[115/182] Checking T1Mapping ...                  [ok]

[116/182] Checking FiberViewerLight ...           [errors]
  homepage, "script does not set 'EXTENSION_HOMEPAGE'"
  iconurl, "script does not set 'EXTENSION_ICONURL'"

[117/182] Checking SPHARM-PDM ...                 [errors]
  homepage, "script does not set 'EXTENSION_HOMEPAGE'"
  iconurl, "script does not set 'EXTENSION_ICONURL'"

[118/182] Checking CleverSeg ...                  [errors]
  homepage, -1, http://slicer.org/slicerWiki/index.php/Documentation/Nightly/Extensions/CleverSeg, <urlopen error timed out>
  iconurl, 404, http://www.example.com/Slicer/Extensions/CleverSeg.png, HTTP Error 404: Not Found
  screenshoturls, 404, http://www.example.com/Slicer/Extensions/CleverSeg/Screenshots/1.png, HTTP Error 404: Not Found

[119/182] Checking DCMQI ...                      [ok]

[120/182] Checking PBNRR ...                      [errors]
  screenshoturls, 404, http://www.example.com/Slicer/Extensions/PBNRR/Screenshots/1.png, HT

[139/182] Checking DICOMwebBrowser ...            [ok]

[140/182] Checking TorchIO ...                    [errors]
  homepage, 404, https://torchio.readthedocs.io/slicer.html, HTTP Error 404: Not Found

[141/182] Checking SlicerOpenCV ...               [ok]

[142/182] Checking MultiLevelRegistration ...     [ok]

[143/182] Checking SlicerElastix ...              [ok]

[144/182] Checking GraphCutSegment ...            [errors]
  iconurl, 404, http://www.example.com/Slicer/Extensions/GraphCutSegment.png, HTTP Error 404: Not Found

[145/182] Checking SlicerPathology ...            [errors]
  screenshoturls, 404, http://wiki.na-mic.org/Wiki/images/3/3b/SlicerPathologybeginworkweek2016.png, HTTP Error 404: Not Found

[146/182] Checking SlicerLayoutButtons ...        [ok]

[147/182] Checking MeshStatisticsExtension ...    [ok]

[148/182] Checking AnomalousFiltersExtension ...  [errors]
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/6/64/MRI_raw.png https://www.slicer.org/sli

[161/182] Checking EasyClip ...                   [ok]

[162/182] Checking DSCMRIAnalysis ...             [ok]

[163/182] Checking Slicer-AirwaySegmentation ...  [ok]

[164/182] Checking LongitudinalPETCT ...          [ok]

[165/182] Checking SegmentEditorExtraEffects ...  [errors]
  screenshoturls, ("script sets 'EXTENSION_SCREENSHOTURLS' to an invalid url [\\\nhttps://raw.githubusercontent.com/lassoan/SlicerSegmentEditorExtraEffects/master/FastMarchingInput.png \\\nhttps://raw.githubusercontent.com/lassoan/SlicerSegmentEditorExtraEffects/master/FastMarchingOutput.png \\\nhttps://raw.githubusercontent.com/lassoan/SlicerSegmentEditorExtraEffects/master/WatershedOutput.png \\]",)

[166/182] Checking ImageCompare ...               [errors]
  homepage, 404, https://www.slicer.org/wiki/Documentation/Nightly/Extensions/ImageCompare/SyntheticCTEvaluation, HTTP Error 404: Not Found

[167/182] Checking OpenCVExample ...              [ok]

[168/182] Checking T2mapping ...                  [erro

[177/182] Checking SegmentationWizard ...         [ok]

[178/182] Checking FilmDosimetryAnalysis ...      [errors]
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/8c/FilmDosimetry_1.0_Gamma.png https://www.slicer.org/slicerWiki/images/7/72/FilmDosimetry_1.0_Registration.png https://www.slicer.org/slicerWiki/images/4/4c/FilmDosimetry_1.0_CalibrationFunction.png https://www.slicer.org/slicerWiki/images/d/d8/FilmDosimetry_1.0_CalibrationBatch.png, HTTP Error 404: Not Found
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/8c/FilmDosimetry_1.0_Gamma.png https://www.slicer.org/slicerWiki/images/7/72/FilmDosimetry_1.0_Registration.png https://www.slicer.org/slicerWiki/images/4/4c/FilmDosimetry_1.0_CalibrationFunction.png https://www.slicer.org/slicerWiki/images/d/d8/FilmDosimetry_1.0_CalibrationBatch.png, HTTP Error 404: Not Found
  screenshoturls, 404, https://www.slicer.org/slicerWiki/images/8/8c/FilmDosimetry_1.0_Gamma.png https://www.slicer.org/slicerWik